## Part 1 - Import and Explore Data 

### 홈 어드밴티지 및 환경 변수 확인

NFL 경기 기록 데이터를 통해   
홈 경기였을 때 상대팀과의 득점 차이가 통계적으로 유의한지 확인한다.   
홈 경기들의 득점이 시즌 및 날씨 환경에 영향을 받는지 확인한다. 

1)  Import the “nfl_game.csv” data file and name the dataframe as “NFL_Game” in Jupyter Notebook.

Descriptions of selected variables

    - Unit of measurement of weather variables:
      temperature - degree Fahrenheit;
      humidity – relative percentage;
      wind – mph
    
    - The variable “score” is the score earned by the team specified in the “team” variable. The variable “opponent_score” is the score earned by the team specified in the “opponent” variable.
    
    - The variable “score_diff” is defined as “score – opponent_score” for the given team.
    

2)  Use the “describe” function to calculate summary statistics for the “date” variable. Use the “describe” function to calculate summary statistics for the “score” variable based on whether it is a home or an away game for the team.

3) Find the correlation efficients between the following pairs of variables:
    - “win” and “home”       
    - “score_diff” and “home”
    - “score” and “weather_temperature”
    - “score” and “weather_humidity”       
    - “score” and “weather_wind_mph”

In [1]:
import pandas as pd
import numpy as np
import datetime
import statsmodels.formula.api as sm

In [2]:
NFL_Game=pd.read_csv("Assignment/Assignment Data/Week 4/nfl_game.csv")
display(NFL_Game)

,game_id,stadium,date,score,weather_temperature,weather_wind_mph,weather_humidity,score_diff,home,win,...,stadium_close,stadium_type,stadium_age,conference,division,season,team_division,team_division_pre2002,team,stadium_neutral
0,1,Orange Bowl,9/2/1966,14,83,6,71,-9,1,0,...,2008.0,outdoor,29,NFC,NFC West,1966,AFC East,AFC East,Miami Dolphins,0
1,1,Orange Bowl,9/2/1966,23,83,6,71,9,0,1,...,2008.0,outdoor,29,AFC,AFC Central,1966,AFC West,AFC West,Oakland Raiders,0
2,2,Rice Stadium,9/3/1966,45,81,7,70,38,1,1,...,NaN,outdoor,16,NFC,NFC West,1966,NaN,AFC Central,Houston Oilers,0
3,2,Rice Stadium,9/3/1966,7,81,7,70,-38,0,0,...,NaN,outdoor,16,NFC,NFC West,1966,AFC West,AFC West,Denver Broncos,0
4,3,Balboa Stadium,9/4/1966,27,70,7,82,20,1,1,...,NaN,outdoor,52,NFC,NFC West,1966,AFC West,AFC West,San Diego Chargers,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24309,12155,MetLife Stadium,12/29/2019,17,42,5,69,-17,1,0,...,NaN,outdoor,9,NFC,NFC East,2019,NFC East,NFC East,New York Giants,0
24310,12156,CenturyLink Field,12/29/2019,26,51,3,70,5,0,1,...,NaN,outdoor,17,AFC,AFC West,2019,NFC West,NFC West,San Francisco 49ers,0
24311,12156,CenturyLink Field,12/29/2019,21,51,3,70,-5,1,0,...,NaN,outdoor,17,AFC,AFC West,2019,NFC West,AFC West,Seattle Seahawks,0
24312,12157,Raymond James Stadium,12/29/2019,28,78,12,77,6,0,1,...,2016.0,outdoor,21,NFC,AFC East,2019,NFC South,NFC West,Atlanta Falcons,0


In [3]:
NFL_Game.shape

(24314, 28)

In [4]:
NFL_Game['date']=pd.to_datetime(NFL_Game['date'])
NFL_Game['date'].describe()

<ipython-input-4-0e929360b896>:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  NFL_Game['date'].describe()


count                   24314
unique                   2065
top       2017-01-01 00:00:00
freq                       32
first     1966-09-02 00:00:00
last      2019-12-29 00:00:00
Name: date, dtype: object

In [5]:
NFL_Game.groupby(['home'])['score'].describe()

,count,mean,std,min,25%,50%,75%,max
home,,,,,,,,
0,12157.0,19.643004,10.166614,0.0,13.0,20.0,27.0,62.0
1,12157.0,22.254997,10.533005,0.0,14.0,21.0,29.0,72.0


5.Calculate correlation coefficients
- win and home
- score_diff and home
- score and weather_temperature
- score and weather_humidity
- score and weather_wind_mph


In [6]:
NFL_Game['win'].corr(NFL_Game['home'])

0.14790211753177582

In [7]:
NFL_Game['score_diff'].corr(NFL_Game['home'])

0.1725059685987271

In [8]:
NFL_Game['score'].corr(NFL_Game['weather_temperature'])

0.03361690080893201

In [9]:
NFL_Game['score'].corr(NFL_Game['weather_humidity'])

-0.03278832087607436

In [10]:
NFL_Game['score'].corr(NFL_Game['weather_wind_mph'])

-0.07895602955806053

## Part 2 - Regression Analysis 1 - Test of Home Game Advantage

In this regression analysis, you will try to determine if playing at home gives teams any advantage in their performance.

Run the following regressions where the variable “score_diff” is the dependent variable.

- Reg1_1: Include a single variable “home” as the independent variable.

In [11]:
Reg1_1 = sm.ols(formula = 'score_diff ~ home', data= NFL_Game).fit()

In [12]:
print(Reg1_1.summary())

                            OLS Regression Results                            
Dep. Variable:             score_diff   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.030
Method:                 Least Squares   F-statistic:                     745.7
Date:                Tue, 06 Jul 2021   Prob (F-statistic):          9.64e-162
Time:                        02:29:39   Log-Likelihood:            -1.0020e+05
No. Observations:               24314   AIC:                         2.004e+05
Df Residuals:                   24312   BIC:                         2.004e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.6120      0.135    -19.309      0.0

Reg1_2
- Dependent variable: score_diff
- Independent variables: home, stadium_capacity, stadium_neutral, the interaction between home and stadium_neutral, team, and opponent

## Part 2 - Regression Analysis 2 -- Impact of Outside Factors on Scores

In this regression analysis, you will investigate if the final score earned by each team is affected by outside factors such as the size and condition of the stadium as well as weather conditions at the stadium.

Run the following regressions where the variable “score” is the dependent variable.

- Reg2_1: Include “season” and “home” variables as independent variables.
- Reg2_2: Include “season,” “home,” “weather_temperature,” “weather_wind_mph,” and “weather_humidity” as independent variables. 

In [17]:
Reg2_1 = sm.ols(formula = 'score ~ season+home', data= NFL_Game).fit()

In [18]:
print(Reg2_1.summary())

                            OLS Regression Results                            
Dep. Variable:                  score   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     273.3
Date:                Tue, 06 Jul 2021   Prob (F-statistic):          4.37e-118
Time:                        02:29:39   Log-Likelihood:                -91246.
No. Observations:               24314   AIC:                         1.825e+05
Df Residuals:                   24311   BIC:                         1.825e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -88.2969      8.615    -10.249      0.0

10.Reg2_2
- Dependent variable: score
- Independent variables: season, home, weather_temperature, weather_wind_mph, and weather_humidity

In [19]:
Reg2_2 = sm.ols(formula = 'score ~ season+home+weather_temperature+weather_wind_mph+weather_humidity', data= NFL_Game).fit()

In [20]:
print(Reg2_2.summary())

                            OLS Regression Results                            
Dep. Variable:                  score   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     129.9
Date:                Tue, 06 Jul 2021   Prob (F-statistic):          2.60e-136
Time:                        02:29:39   Log-Likelihood:                -91195.
No. Observations:               24314   AIC:                         1.824e+05
Df Residuals:                   24308   BIC:                         1.825e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept             -58.1972    

11.Reg2_3
- Dependent variable: score
- Independent variables: season, home, weather_temperature, weather_wind_mph, and weather_humidity, stadium_capacity,  stadium_age, stadium_type, stadium_neutral, and the interaction between home and stadium_neutral